# Урок 4. Парсинг HTML. XPath

1)Написать приложение, которое собирает основные новости с сайтов mail.ru и lenta.ru.

2) (Дополнительно) Собрать все новости со страницы https://yandex.ru/news/

Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

In [1]:
import requests as req
import pandas as pd
from lxml import html
from datetime import datetime, timedelta
import locale

**user_agent_mob** - для запроса к мобильной версии яндекс новостей  
**locale.setlocale(locale.LC_TIME, '')** - для обработки даты публикации(месяц написан буквами) с ленты

In [2]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
user_agent_mob = 'Mozilla/5.0 (Linux; Android 5.1.1; Nexus 5 Build/LMY48B; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/43.0.2357.65 Mobile Safari/537.36'
locale.setlocale(locale.LC_TIME, '')

'ru_RU.UTF-8'

**ma_params, le_params, ya_params**  
индивидуальные параметры для каждого сайта

In [3]:
ma_params = {'headers' : {'User-Agent': user_agent},
             'link' : 'https://m.mail.ru/',
             'news_block' : '//div[@id="news-0"]',
             'href' : './/a[contains(@class,"list__item")]/@href',
             'href_prefix' : '',
             'title' : './/span[contains(@class,"list__item__title")]/text()',
             'datetime' : './/*[@class="note__text breadcrumbs__text js-ago"]/@datetime',
             'datetime_format' : ['%Y-%m-%d%H:%M:%S']}

In [4]:
le_params = {'headers' : {'User-Agent': user_agent},
             'link' : 'https://m.lenta.ru/',
             'news_block' : '//ul[@class="b-list b-list_top-7"]',
             'href' : './/a[@class="b-list-item__link"]/@href', 
             'href_prefix' : 'https://m.lenta.ru/',
             'title' : './/span[@class="b-list-item__title"]/text()',
             'datetime' : './/time[@class="g-time"]/@datetime',
             'datetime_format' : [' %H:%M, %d %B %Y']}

In [5]:
ya_params = {'headers' : {'User-Agent': user_agent_mob},
             'link' : 'https://yandex.ru/news/',
             'news_block' : '//div[@class="news-container"]',
             'href' : './/a[contains(@class,"Link link link_")]/@href',
             'href_prefix' : '',
             'title' : './/span[@class="link link_pseudo card__link"]/text()',
             'datetime' : './/span[@class="sport-date"]/text()',
             'datetime_format' : ['%H:%M', '%H:%M:%S%Y-%m-%d']}

**get_news** - получение основной информации  
**get_ma_datetime** - дата-время с мэйла (обходим ссылки с новостями)  
**datetime_format** - дата-время с разных сайтов - это строки в разном формате. преобразуем все к одному виду

In [6]:
class XPath_news_scraper:
    
    def datetime_format(self):
        for i in range(len(self.news['datetime'])): 
            news_datetime = datetime.strptime(self.news['datetime'][i], self.params['datetime_format'][0])
            if news_datetime.year == 1900:
                now = datetime.now()
                new = str(news_datetime.time())
                if now.time() > news_datetime.time():
                    new += str(now.date())
                else:
                    new += str((now - timedelta(days=1)).date())
                news_datetime = datetime.strptime(new, self.params['datetime_format'][1])
            self.news['datetime'][i] = news_datetime
    
    def get_ma_datetime(self):
        datetime = []
        for link in self.news['href']:
            response = req.get(link, headers=self.params['headers'])
            if response.status_code == 200:
                doc = html.fromstring(response.text) 
                try:
                    datetime.append(''.join(doc.xpath(self.params['datetime'])[0].replace('T', '+').split('+')[:-1]))
                except:
                    datetime.append(None)
            else:
                datetime.append(None)
        return datetime
        
    def get_news(self):
        response = req.get(self.params['link'], headers=self.params['headers'])
        if response.status_code == 200:
            doc = html.fromstring(response.text)
            try:
                news_block = doc.xpath(self.params['news_block'])
            except:
                return print('новости не найдены')
            self.news['site'] = self.params['link']
            try:
                self.news['title'] = news_block[0].xpath(self.params['title'])
            except:
                self.news['title'] = None
            try:
                self.news['href'] = [self.params['href_prefix'] + item for item in news_block[0].xpath(self.params['href'])]
            except:
                self.news['href'] = None
            if self.params['link'] != ma_params['link']:
                try:
                    self.news['datetime'] = news_block[0].xpath(self.params['datetime'])
                except:
                    self.news['datetime'] = None        
            else:
                self.news['datetime'] = self.get_ma_datetime()
            self.datetime_format()                
        else:
            print('что-то пошло не так: responce status code =', response.status_code)         
        
    def __init__(self, params):
        self.news = {}
        self.params = params
        self.get_news()

In [7]:
mail_scr = XPath_news_scraper(ma_params)

In [8]:
lenta_scr = XPath_news_scraper(le_params)

In [9]:
yandex_scr = XPath_news_scraper(ya_params)

In [10]:
df = pd.concat([pd.DataFrame(mail_scr.news), pd.DataFrame(lenta_scr.news), pd.DataFrame(yandex_scr.news)], ignore_index=True)

In [11]:
df

,site,title,href,datetime
0,https://m.mail.ru/,США решили блокировать «Северный поток — 2» че...,https://r.mail.ru/n314803699?&test_id=63&rnd=1...,2019-11-24 07:42:40
1,https://m.mail.ru/,Пилот рейса «Москва — Анапа» скончался после э...,https://r.mail.ru/n314814993?&test_id=63&rnd=1...,2019-11-24 12:01:22
2,https://m.mail.ru/,Синоптики рассказали о последствиях арктическо...,https://r.mail.ru/n314810190?&test_id=63&rnd=1...,2019-11-24 10:12:29
3,https://m.mail.ru/,Букву Ё вернули в названия 198 новгородских де...,https://r.mail.ru/n314817650?&test_id=63&rnd=1...,2019-11-24 13:10:29
4,https://m.mail.ru/,Россиян поразило внезапно обнажившееся дно Азо...,https://r.mail.ru/n314806638?&test_id=63&rnd=1...,2019-11-24 08:59:25
5,https://m.mail.ru/,Новый внедорожник для миллиардеров: он похож н...,https://r.mail.ru/n314761702?&test_id=63&rnd=7...,2019-11-22 17:30:45
6,https://m.mail.ru/,"Культовая техника 90-х: вспоминаем, как это бы...",https://r.mail.ru/n314821146?&test_id=63&rnd=1...,2019-11-23 08:29:00
7,https://m.mail.ru/,«Какая красивая Аллочка»: Галкин показал новое...,https://r.mail.ru/n314769752?&test_id=63&rnd=1...,2019-11-23 13:11:43
8,https://m.lenta.ru/,Россиянин попытался отобрать пистолет у полице...,https://m.lenta.ru//news/2019/11/24/gun/,2019-11-24 12:28:00
9,https://m.lenta.ru/,Красная книга России усомнилась в принадлежнос...,https://m.lenta.ru//news/2019/11/24/dagestan_n...,2019-11-24 15:33:00
